<a href="https://colab.research.google.com/github/Swatantra-D/Segmentation/blob/main/OCT_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

####


1.   Fix NT_aft_wnd_d0_t_5 
2.   Fix Nt_befwnd_t1_3 inverse 2 == 3 $ 3 == 2
3.   Nt_d12_2 1 == 3 & 2 == 2
4.   CCF_wd12_4 min == 1 & max == 2
5.   



### import libraries

In [44]:
import os
import sys
import cv2
import PIL
import skimage
import sklearn
import numpy as np
import pandas as pd
import torch

from PIL import Image
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [2]:
maskpath = r"C:\Users\Swata\OneDrive\Desktop\OCT\Mask"
imagepath = r"C:\Users\Swata\OneDrive\Desktop\OCT"

In [3]:
im_width = 1712
im_height = 512

In [4]:
images = os.listdir(imagepath)
masks = os.listdir(maskpath)

In [36]:
IMAGES = np.zeros((len(masks), im_height, im_width, 3), dtype=np.float32)
MASKS = np.zeros((len(masks), im_height, im_width, 1), dtype=np.float32)

for n,image in enumerate(masks):

    img = img_to_array(load_img(r"{}\{}".format(imagepath,image[:-4]+".BMP")))
    img = resize(img, (512, 1712, 3), mode = 'constant', preserve_range = True)           

    mask = img_to_array(load_img(r"{}\{}".format(maskpath,image)))
    mask = resize(mask, (512, 1712, 1), mode = 'constant', preserve_range = True)    

    IMAGES[n] = img/255.0
    MASKS[n] = mask/1.0

In [38]:
IMAGES.shape, MASKS.shape

((46, 512, 1712, 3), (46, 512, 1712, 1))

### Hide em



In [ ]:
C = Image.open(r"{}\{}".format(imagepath,images[0]))
np.asarray(C).shape

(512, 1712, 3)

In [ ]:
from torchvision import transforms

In [ ]:
transform = transforms.ToTensor()

In [ ]:
img_array = np.reshape(IMAGES, (len(IMAGES), 512, 1712, 3))
img_array = img_array.astype('float32') / 255.

C:\Users\Swata\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


ValueError: ignored

In [ ]:
IMAGES[0].shape

(512, 1712, 3)

In [ ]:
img = cv2.resize(IMAGES[0],(1712, 512))

In [ ]:
img.shape

(512, 1712, 3)

### Next


In [41]:
import segmentation_models as sm
from segmentation_models import Unet
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss
from segmentation_models.metrics import iou_score

In [46]:
X_train, X_valid, y_train, y_valid = train_test_split(IMAGES, MASKS, test_size=0.33, random_state=2022)

In [50]:
BACKBONE = 'resnet34'
preprocess_input = get_preprocessing(BACKBONE)
X_train = preprocess_input(X_train)
X_valid = preprocess_input(X_valid)

In [56]:
model = Unet(BACKBONE, encoder_weights=None)
model.compile('Adam', loss="categorical_crossentropy", metrics=[iou_score])

In [57]:
model.fit(
    x=X_train,
    y=y_train,
    batch_size=1,
    epochs=100,
    validation_data=(X_valid, y_valid),
)

Epoch 1/100


ValueError: ignored